In [32]:
import os
import random
import sys
import numpy as np
from safetensors.torch import load_file
import torch
import math
from safetensors.torch import save_file

import time
start_time = time.time()
# ----------------------------------------------------------

method = 'Qwen2_5_0_5B_base'
safetensors_path_act = f'saved_tensor/{method}/activation.safetensors'
state_dict_qwt_input = load_file(safetensors_path_act)

In [33]:
for k, v in state_dict_qwt_input.items():
    print(k, v.shape)

layers.0.input torch.Size([1, 2048, 896])
layers.1.input torch.Size([1, 2048, 896])
layers.10.input torch.Size([1, 2048, 896])
layers.11.input torch.Size([1, 2048, 896])
layers.12.input torch.Size([1, 2048, 896])
layers.13.input torch.Size([1, 2048, 896])
layers.14.input torch.Size([1, 2048, 896])
layers.15.input torch.Size([1, 2048, 896])
layers.16.input torch.Size([1, 2048, 896])
layers.17.input torch.Size([1, 2048, 896])
layers.18.input torch.Size([1, 2048, 896])
layers.19.input torch.Size([1, 2048, 896])
layers.2.input torch.Size([1, 2048, 896])
layers.20.input torch.Size([1, 2048, 896])
layers.21.input torch.Size([1, 2048, 896])
layers.22.input torch.Size([1, 2048, 896])
layers.23.input torch.Size([1, 2048, 896])
layers.3.input torch.Size([1, 2048, 896])
layers.4.input torch.Size([1, 2048, 896])
layers.5.input torch.Size([1, 2048, 896])
layers.6.input torch.Size([1, 2048, 896])
layers.7.input torch.Size([1, 2048, 896])
layers.8.input torch.Size([1, 2048, 896])
layers.9.input torch

In [ ]:
import torch
import torch.nn as nn

# 实例化MSELoss对象
loss = nn.MSELoss()

In [ ]:
for a, b in zip(state_dict_base.values(), state_dict_qwt_input.values()):
    print(a.equal(b)) #, a.view(-1)[:5], b.view(-1)[:5])

In [ ]:
with open('log/123.log', 'w') as f:
    for idx, (a, b, c, d, e) in enumerate(zip(state_dict_base.values(), state_dict_quant.values(), state_dict_base.values(), state_dict_clamp.values(), state_dict_clamp_qwt.values())):
        f.writelines(f'{idx} base-quant: {loss(a.float(), b.float()).item():>8.4f}, base-clamp: {loss(c.float(), d.float()).item():>8.4f}, base-qwt: {loss(a.float(), e.float()).item():>8.4f}\n') # 11, 16

In [ ]:
for idx in range(22): 
    print(idx, state_dict_base[f'base_layers.{idx}.input'].equal(state_dict_qwt[f'qwt_layers.{idx}.input_full']))
    # print(idx, loss(state_dict_base[f'base_layers.{idx}.input'].float(), state_dict_qwt[f'qwt_layers.{idx}.input_full'].float()))
    # print('='*10)
    # print(state_dict_base[f'base_layers.{idx}.input'].view(-1)[:5])
    # print(state_dict_qwt[f'qwt_layers.{idx}.input_full'].view(-1)[:5])


In [ ]:
def tensor_gpu_memory(tensor, name=None):
    if tensor.is_cuda:
        size_in_bytes = tensor.element_size() * tensor.numel()
        size_in_MB = size_in_bytes / 1024**2
        print(f"{name if name else ''} size: {size_in_MB:.2f} MB")
    else:
        print(f"{name if name else ''} is not on GPU.")

In [ ]:
import torch
tensor = torch.empty((512, 2048, 4096), dtype=torch.float32).to('cuda')
# print((4 * 512 * 2048 * 4096 * 32) / 1024 / 1024 / 1024 / 1024)

In [ ]:
tensor_gpu_memory(tensor)